Script para generar un csv a partir de los archivos html que componen las páginas de supermercado de Carrefour

In [ ]:
import os
import json
import csv
import re
from pathlib import Path

def extraer_productos_de_html(ruta_archivo):
    """
    Extrae los productos del array 'impressions' dentro del archivo HTML
    """
    try:
        with open(ruta_archivo, 'r', encoding='utf-8') as f:
            contenido = f.read()
        
        # Buscar el patrón window["impressions"]=[...]
        patron = r'window\["impressions"\]\s*=\s*(\[.*?\]);'
        match = re.search(patron, contenido, re.DOTALL)
        
        if match:
            json_str = match.group(1)
            productos = json.loads(json_str)
            return productos
        else:
            print(f"No se encontraron productos en: {ruta_archivo}")
            return []
    
    except Exception as e:
        print(f"Error al procesar {ruta_archivo}: {str(e)}")
        return []

def procesar_carpetas(carpeta_madre, archivo_salida):
    """
    Procesa todos los archivos HTML en la carpeta madre y sus subcarpetas
    """
    todos_productos = []
    campos_unicos = set()
    
    # Recorrer todos los archivos HTML
    carpeta_path = Path(carpeta_madre)
    archivos_html = list(carpeta_path.rglob('*.html')) + list(carpeta_path.rglob('*.htm'))
    
    print(f"Encontrados {len(archivos_html)} archivos HTML")
    
    for idx, archivo in enumerate(archivos_html, 1):
        print(f"Procesando ({idx}/{len(archivos_html)}): {archivo}")
        productos = extraer_productos_de_html(archivo)
        
        # Añadir información del archivo de origen
        for producto in productos:
            producto['archivo_origen'] = str(archivo.relative_to(carpeta_madre))
            producto['carpeta_origen'] = str(archivo.parent.relative_to(carpeta_madre))
            
            # Recopilar todos los campos únicos
            campos_unicos.update(producto.keys())
        
        todos_productos.extend(productos)
    
    print(f"\nTotal de productos extraídos: {len(todos_productos)}")
    
    # Ordenar campos para el CSV (campos importantes primero)
    campos_prioritarios = ['item_name', 'item_id', 'item_ean', 'price', 'currency', 
                          'item_brand', 'item_category', 'quantity', 'discount']
    
    campos_ordenados = []
    for campo in campos_prioritarios:
        if campo in campos_unicos:
            campos_ordenados.append(campo)
            campos_unicos.remove(campo)
    
    # Añadir el resto de campos alfabéticamente
    campos_ordenados.extend(sorted(campos_unicos))
    
    # Escribir CSV
    with open(archivo_salida, 'w', newline='', encoding='utf-8-sig') as f:
        writer = csv.DictWriter(f, fieldnames=campos_ordenados, extrasaction='ignore')
        writer.writeheader()
        writer.writerows(todos_productos)
    
    print(f"\nArchivo CSV generado: {archivo_salida}")
    print(f"Campos incluidos: {len(campos_ordenados)}")
    print(f"Productos guardados: {len(todos_productos)}")

if __name__ == "__main__":
    # Configuración
    CARPETA_MADRE = "Archivos_html"
    ARCHIVO_SALIDA = "carrefour_raw.csv"
    
    # Verificar que existe la carpeta
    if not os.path.exists(CARPETA_MADRE):
        print(f"ERROR: La carpeta '{CARPETA_MADRE}' no existe")
        print(f"Ruta buscada: {os.path.abspath(CARPETA_MADRE)}")
    else:
        print(f"Procesando carpeta: {os.path.abspath(CARPETA_MADRE)}\n")
        procesar_carpetas(CARPETA_MADRE, ARCHIVO_SALIDA)
        print("\n¡Proceso completado!")

SyntaxError: (unicode error) 'unicodeescape' codec can't decode bytes in position 2-3: truncated \UXXXXXXXX escape (1524866538.py, line 85)